In [ ]:
!pip install streamlit -q
!pip install streamlit_option_menu -q
!pip install streamlit pyngrok -q
!pip install --upgrade streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 29.4 MB/s eta 0:00:00


In [ ]:
%%writefile main.py
import streamlit as st
import pandas as pd
import numpy as np
from streamlit_option_menu import option_menu
import csv
from datetime import datetime
import hashlib

st.set_page_config(page_title="UPI FRAUD DETECTION",page_icon="/content/logo.png")

def streamlit_menu():
    selected = option_menu(
        menu_title=None,
        options=["Home", "Analysis", "Result", "Login", "Team Details"],
        icons=["house", "bar-chart", "clipboard-check", "person-circle", "people"],
        menu_icon="cast",
        default_index=0,
        orientation="horizontal",
        styles={
            "container": {"padding": "0!important", "background-color": "#FFFFFF"},
            "icon": {"color": "blue", "font-size": "25px"},
            "nav-link": {
                "font-size": "18px",
                "text-align": "center",
                "margin": "0px",
                "--hover-color": "#eee",
            },
            "nav-link-selected": {"background-color": "white"},
        },
    )
    return selected

selected = streamlit_menu()

st.markdown("""
    <style>
    .stButton>button {
        background-color: blue;
        color: white;
        width: 100%;
        font-size: 18px;
        padding: 10px;
        border: none;
        border-radius: 5px;
    }
    .stButton>button:hover {
        background-color: darkblue;
    }
    </style>
    """, unsafe_allow_html=True)

if selected=="Home":
    st.markdown('<p style="color: black; text-align: center; font-family: Segoe Fluent Icons; font-size : 38px">UPI FRAUD DETECTION</p>',unsafe_allow_html=True)
    st.markdown('<p style="color: black; text-align: center; font-family: Segoe Fluent Icons; font-size : 20px">Enter your Details below to Check for Potential Fraud.</p>',unsafe_allow_html=True)

    upi_container=st.container()
    default_date = datetime.today().date()

    if 'user_data' not in st.session_state:
        st.session_state.user_data = {}

    def save_to_csv(data):
        fieldnames = data.keys()
        with open("user_data.csv", mode="a", newline="") as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writerow(data)

    with upi_container:

        with st.form("UPI Form"):
          col1, col2 = st.columns(2)
          amount = col1.number_input('Transaction Amount (₹):', min_value=1)
          nameOrig = col2.text_input('Your Name (Initiator):')
          col3, col4 = st.columns(2)
          oldbalanceOrg = col3.number_input('Your Balance Before Transaction (₹):', min_value=0.0)
          newbalanceOrig = col4.number_input('Your Balance After Transaction (₹):', min_value=0.0)
          col5, col6 = st.columns(2)
          nameDest = col5.text_input('Recipient Name:')
          oldbalanceDest = col6.number_input('Recipient’s Balance Before Transaction (₹):', min_value=0.0)
          col7, col8 = st.columns(2)
          newbalanceDest = col7.number_input('Recipient’s Balance After Transaction (₹):', min_value=0.0)
          payment_type = col8.selectbox('Type of Transaction:', ['CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER'])

          submit_button=st.form_submit_button(label='Predict')
        if submit_button:
          st.write('Transaction Amount (₹): ', amount)
          st.write('Your Name (Initiator): ', nameOrig)
          st.write('Your Balance Before Transaction (₹):', oldbalanceOrg)
          st.write('Your Balance After Transaction (₹):', newbalanceOrig)
          st.write('Recipient Name: ', nameDest)
          st.write('Recipient’s Balance Before Transaction (₹): ', oldbalanceDest)
          st.write('Recipient’s Balance After Transaction (₹): ', newbalanceDest)
          st.write('Type of Transaction: ', payment_type)

          st.session_state.user_data = {
              "transaction_amount": amount,
              "initiator_name": nameOrig,
              "initiator_balance_before": oldbalanceOrg,
              "initiator_balance_after": newbalanceOrig,
              "recipient_name": nameDest,
              "recipient_balance_before": oldbalanceDest,
              "recipient_balance_after": newbalanceDest,
              "transaction_type": payment_type
          }

          save_to_csv(st.session_state.user_data)

elif selected=="Analysis":
     st.markdown('<p style="color: black; text-align: center; font-family: Segoe Fluent Icons; font-size : 30px">Analysis</p>',unsafe_allow_html=True)

elif selected=="Result":
    st.markdown('<p style="color: black; text-align: center; font-family: Segoe Fluent Icons; font-size : 30px">Result</p>',unsafe_allow_html=True)

elif selected=="Login":

    def save_to_csv(data):
        fieldnames=['username', 'password']
        with open("user_login.csv", mode="a", newline="") as file:
            writer=csv.DictWriter(file, fieldnames=fieldnames)
            writer.writerow(data)

    if 'user_db' not in st.session_state:
        st.session_state.user_db={}

    def hash_password(password):
        return hashlib.sha256(password.encode()).hexdigest()

    def check_user(username):
        return username in st.session_state.user_db

    def verify_password(username, password):
        return st.session_state.user_db[username]==hash_password(password)

    login_container=st.container()
    register_container=st.container()

    def login_page():
        with login_container:
            with st.form("Login Form"):
                st.markdown("""
                <style>
                .stButton>button {
                    background-color: red;
                    color: white;
                    width: 100%;
                    font-size: 18px;
                    padding: 10px;
                    border: none;
                    border-radius: 5px;
                }
                .stButton>button:hover {
                    background-color: darkred;
                }
                </style>
                """, unsafe_allow_html=True)
                st.markdown('<p style="color: black; text-align: center; font-family: Segoe Fluent Icons; font-size: 30px">Login</p>', unsafe_allow_html=True)
                username=st.text_input("Username")
                password=st.text_input("Password", type="password")
                submit_button=st.form_submit_button("Login")

            if submit_button:
                if check_user(username):
                    if verify_password(username, password):
                        st.success(f"Welcome back, {username}!")
                    else:
                        st.error("Incorrect Password. Please try again.")
                else:
                    st.error("Username does not exist. Please register yourself.")


            if st.button("Register an Account"):
                st.session_state.page="register"

    def register_page():
        with register_container:
            with st.form("Register Form"):
                st.markdown("""
                <style>
                .stButton>button {
                    background-color: red;
                    color: white;
                    width: 100%;
                    font-size: 18px;
                    padding: 10px;
                    border: none;
                    border-radius: 5px;
                }
                .stButton>button:hover {
                    background-color: darkred;
                }
                </style>
                """, unsafe_allow_html=True)
                st.markdown('<p style="color: black; text-align: center; font-family: Segoe Fluent Icons; font-size: 30px">Register an Account</p>', unsafe_allow_html=True)
                username=st.text_input("Username")
                password=st.text_input("Password", type="password")
                confirm_password=st.text_input("Confirm Password", type="password")
                submit_button=st.form_submit_button("Register")

            if submit_button:
                if username in st.session_state.user_db:
                    st.error("Username already exists. Please choose a different one.")
                elif password!=confirm_password:
                    st.error("Password does not match. Please try again.")
                else:
                    st.session_state.user_db[username]=hash_password(password)
                    st.success(f"Account created successfully! You can now Login.")
                    save_to_csv({'username': username, 'password': password})

                    st.session_state.page="login"


            if st.button("Back to Login"):
                st.session_state.page="login"

    def main():
        if 'page' not in st.session_state:
            st.session_state.page="login"

        if st.session_state.page=="login":
            login_page()
        elif st.session_state.page=="register":
            register_page()

    if __name__=="__main__":
        main()

elif selected=="Team Details":
    st.markdown('<p style="color: black; text-align: center; font-family: Segoe Fluent Icons; font-size : 30px">Team Details</p>',unsafe_allow_html=True)
    team_container=st.container()
    with team_container:
        st.markdown("""
            <div style="border: 2px solid #000; padding: 10px; margin: 10px; border-radius: 10px;">
                <p style="color: black; text-align: center; font-family: Segoe Fluent Icons; font-size: 24px;">
                    <span style="display: inline-block; width: 200px; text-align: left;">2021UEA6571</span>
                    <span style="display: inline-block; width: 250px; text-align: left;">Sarli</span>
                    <span style="display: inline-block; width: 100px; text-align: left;">ECAM-2</span>
                </p>
            </div>
        """, unsafe_allow_html=True)

        st.markdown("""
            <div style="border: 2px solid #000; padding: 10px; margin: 10px; border-radius: 10px;">
                <p style="color: black; text-align: center; font-family: Segoe Fluent Icons; font-size: 24px;">
                    <span style="display: inline-block; width: 200px; text-align: left;">2021UEA6617</span>
                    <span style="display: inline-block; width: 250px; text-align: left;">Kaushal Asiwal</span>
                    <span style="display: inline-block; width: 100px; text-align: left;">ECAM-2</span>
                </p>
            </div>
        """, unsafe_allow_html=True)

        st.markdown("""
            <div style="border: 2px solid #000; padding: 10px; margin: 10px; border-radius: 10px;">
                <p style="color: black; text-align: center; font-family: Segoe Fluent Icons; font-size: 24px;">
                    <span style="display: inline-block; width: 200px; text-align: left;">2021UEA7024</span>
                    <span style="display: inline-block; width: 250px; text-align: left;">Prem Dev Singh</span>
                    <span style="display: inline-block; width: 100px; text-align: left;">ECAM-2</span>
                </p>
            </div>
        """, unsafe_allow_html=True)

        st.markdown("""
            <div style="border: 2px solid #000; padding: 10px; margin: 10px; border-radius: 10px;">
                <p style="color: black; text-align: center; font-family: Segoe Fluent Icons; font-size: 24px;">
                    <span style="display: inline-block; width: 200px; text-align: left;">2021UEA7050</span>
                    <span style="display: inline-block; width: 250px; text-align: left;">Aditya Pandey</span>
                    <span style="display: inline-block; width: 100px; text-align: left;">ECAM-2</span>
                </p>
            </div>
        """, unsafe_allow_html=True)

footer_html="""
<div style='text-align: center; color: black;'>
<p>2024 UPI FRAUD DETECTION.  All Rights Reseverd.<p>
<p>© Designed and Developed by BTP Team.</p>
</div>
"""
st.markdown(footer_html,unsafe_allow_html=True)

Writing main.py


In [ ]:
from pyngrok import ngrok
import os


# List active tunnels
#tunnels = ngrok.get_tunnels()

# Terminate an existing tunnel
#ngrok.disconnect(tunnels[0].public_url)  # Disconnect the first active tunnel

ngrok.set_auth_token("2oc5YxY57luUqljPoT7NxCM4Ymr_3YmhNCxSPGaN4kUAkWri3")

port = 8501

os.system(f"streamlit run main.py &")

public_url = ngrok.connect(port)
print(f"Streamlit app running at: {public_url}")

Streamlit app running at: NgrokTunnel: "https://2d3f-34-48-76-189.ngrok-free.app" -> "http://localhost:8501"
